In [1]:
import pandas as pd
import numpy as np

import statsmodels.api as sm

# plt.style.use('dark_background')
random_state=42

In [2]:
tesis = pd.read_csv("../Datos/tesis_final_preprocesado.csv")
tesis.head()

,idPaciente,Mes,TAS,Adherencia,Sexo,Edad,tas_basal,Adherencia_Acumulada,Adherencia_Total,Adherencia_Perfecta,TAS_Media_Acumulada,Intercept,Adherencia_lag1,TAS_lag1,Adherencia_Acumulada_lag2,TAS_Media_Acumulada_lag2,Mes*Adherencia,Mes*Adherencia_Perfecta
0,4026,1,119.000000,1,0,76.0,116,1.0,1.0,True,119.000000,1,0.0,116.00000,0.0,116.000000,1,1
1,4026,2,127.000000,1,0,76.0,116,1.0,1.0,True,123.000000,1,1.0,119.00000,0.0,116.000000,2,2
2,4026,3,140.000000,1,0,76.0,116,1.0,1.0,True,128.666667,1,1.0,127.00000,1.0,119.000000,3,3
3,4026,4,146.712710,1,0,76.0,116,1.0,1.0,True,133.178178,1,1.0,140.00000,1.0,123.000000,4,4
4,4026,5,177.708084,1,0,76.0,116,1.0,1.0,True,142.084159,1,1.0,146.71271,1.0,128.666667,5,5


In [3]:
fixed_effects = [
    "Intercept",
    "Sexo",
    "Edad",
    "Mes",
]

In [4]:
mixed_both = sm.MixedLM(
    endog=tesis["TAS"],
    exog=tesis[["Intercept", "Sexo", "Edad", "Adherencia", "Mes", "Mes*Adherencia"]],
    groups=tesis["idPaciente"],
    exog_re=tesis[["Intercept", "Mes"]]
).fit(reml=True)

print(mixed_both.summary())

              Mixed Linear Model Regression Results
Model:                MixedLM   Dependent Variable:   TAS        
No. Observations:     3920      Method:               REML       
No. Groups:           560       Scale:                113.2865   
Min. group size:      7         Log-Likelihood:       -15393.9298
Max. group size:      7         Converged:            Yes        
Mean group size:      7.0                                        
-----------------------------------------------------------------
                     Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
-----------------------------------------------------------------
Intercept           122.271    2.508 48.751 0.000 117.355 127.186
Sexo                  3.760    0.749  5.022 0.000   2.292   5.227
Edad                  0.167    0.039  4.344 0.000   0.092   0.243
Adherencia           -1.525    1.144 -1.333 0.183  -3.767   0.717
Mes                   0.371    0.240  1.545 0.122  -0.100   0.841
Mes*Adherencia       -0.

In [5]:
####################
# HECHO EN R PORQUE NO CONVERGE
####################


# mixed_time = sm.MixedLM.from_formula(
#     "TAS ~ Sexo + Edad + Adherencia + Mes + Adherencia*Mes",
#     data=tesis,
#     groups="idPaciente",
#     vc_formula={"Mes": "0 + Mes"}
# ).fit(reml=True, maxiter=100000)

# print(mixed_time.summary())
# print(-2*mixed_time.llf, -2*mixed_both.llf, (-2*mixed_time.llf)-(-2*mixed_both.llf))

In [6]:
mixed_intercept = sm.MixedLM(
    endog=tesis["TAS"],
    exog=tesis[["Intercept", "Sexo", "Edad", "Adherencia", "Mes", "Mes*Adherencia"]],
    groups=tesis["idPaciente"],
    exog_re=tesis["Intercept"]
).fit(reml=True)

print(mixed_intercept.summary())
print(-2*mixed_intercept.llf, -2*mixed_both.llf, (-2*mixed_intercept.llf)-(-2*mixed_both.llf))

           Mixed Linear Model Regression Results
Model:             MixedLM  Dependent Variable:  TAS        
No. Observations:  3920     Method:              REML       
No. Groups:        560      Scale:               123.3531   
Min. group size:   7        Log-Likelihood:      -15419.4875
Max. group size:   7        Converged:           Yes        
Mean group size:   7.0                                      
------------------------------------------------------------
                Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
------------------------------------------------------------
Intercept      122.102    2.482 49.204 0.000 117.238 126.966
Sexo             3.771    0.752  5.016 0.000   2.297   5.244
Edad             0.165    0.039  4.261 0.000   0.089   0.241
Adherencia      -1.129    1.086 -1.039 0.299  -3.257   1.000
Mes              0.430    0.224  1.924 0.054  -0.008   0.868
Mes*Adherencia  -0.866    0.250 -3.467 0.001  -1.355  -0.376
Intercept Var   61.168    0.459     